# Fake News Detector

In [14]:
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm


In [ ]:
true_dataset = pd.read_csv('data/True.csv')
fake_dataset = pd.read_csv('data/Fake.csv')

In [ ]:
from hazm import Normalizer, SentenceTokenizer, word_tokenize
normalizer = Normalizer()
sent_tokenizer = SentenceTokenizer()

In [ ]:


true_dataset['title'] = true_dataset.apply(lambda row: row['title'].lower(), axis=1)
fake_dataset['title'] = fake_dataset.apply(lambda row: row['title'].lower(), axis=1)

true_dataset['text'] = true_dataset.apply(lambda row: row['text'].lower(), axis=1)
fake_dataset['text'] = fake_dataset.apply(lambda row: row['text'].lower(), axis=1)

In [ ]:
true_dataset['text']

In [ ]:
true_dataset['text'] = true_dataset.apply(lambda row: normalizer.normalize(row['text']), axis=1)

true_dataset['title'] = true_dataset.apply(lambda row: normalizer.normalize(row['title']), axis=1)

fake_dataset['text'] = fake_dataset.apply(lambda row: normalizer.normalize(row['text']), axis=1)

fake_dataset['title'] = fake_dataset.apply(lambda row: normalizer.normalize(row['title']), axis=1)

In [ ]:
# true_dataset.to_csv('data/TrueV1.csv', index=False)
# fake_dataset.to_csv('data/FakeV1.csv', index=False)

In [ ]:
# true_dataset = pd.read_csv('data/TrueV1.csv')
# fake_dataset = pd.read_csv('data/FakeV1.csv')

In [ ]:
def remove_numbers(text):
    return str(re.sub(r'\d+', '', text))

def remove_urls(text):
    return str(re.sub(r'http[s]?://\S+|www\.\S+', '', text))

### Remove num and url

In [ ]:
true_dataset['text'] = true_dataset.apply(lambda row: remove_numbers(remove_urls(str(row['text']))), axis=1)

true_dataset['title'] = true_dataset.apply(lambda row: remove_numbers(remove_urls(str(row['title']))), axis=1)

fake_dataset['text'] = fake_dataset.apply(lambda row: remove_numbers(remove_urls(str(row['text']))), axis=1)

fake_dataset['title'] = fake_dataset.apply(lambda row: remove_numbers(remove_urls(str(row['title']))), axis=1)

### Remove puncs

In [ ]:
true_dataset['text'] = true_dataset.apply(lambda row: re.sub(r'[^\w\s]', '', row["text"]), axis=1)

true_dataset['title'] = true_dataset.apply(lambda row: re.sub(r'[^\w\s]', '', row["title"]), axis=1)

fake_dataset['text'] = fake_dataset.apply(lambda row: re.sub(r'[^\w\s]', '', row["text"]), axis=1)

fake_dataset['title'] = fake_dataset.apply(lambda row: re.sub(r'[^\w\s]', '', row["title"]), axis=1)

In [ ]:
fake_dataset

In [ ]:
# true_dataset.to_csv('data/TrueV2.csv', index=False)
# fake_dataset.to_csv('data/FakeV2.csv', index=False)

### Tokenize

In [ ]:
# true_dataset['text'] = true_dataset.apply(lambda row: word_tokenize(row['text']), axis=1)

# true_dataset['title'] = true_dataset.apply(lambda row: word_tokenize(row['title']), axis=1)

# fake_dataset['text'] = fake_dataset.apply(lambda row: word_tokenize(row['text']), axis=1)

# fake_dataset['title'] = fake_dataset.apply(lambda row: word_tokenize(row['title']), axis=1)

# Training KNN

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer()


In [ ]:
true_dataset = pd.read_csv('data/TrueV2.csv')
fake_dataset = pd.read_csv('data/FakeV2.csv')

In [ ]:
true_dataset['label'] = 1
fake_dataset['label'] = 0
dataset = pd.concat([true_dataset, fake_dataset], ignore_index=True)

In [12]:
dataset["text"] = dataset["text"].astype(str).apply(word_tokenize)
dataset["title"] = dataset["text"].astype(str).apply(word_tokenize)

In [15]:
for series in [dataset['text'], dataset['title']]: 
    for row in tqdm(series, total=series.shape[0], desc="Fitting tokenizer"):
        tokenizer.fit_on_texts(row)

Fitting tokenizer:  42%|████▏     | 18939/44898 [09:50<13:29, 32.07it/s]


KeyboardInterrupt: 

In [13]:
dataset["text_sequences"] = dataset["text"].apply(tokenizer.texts_to_sequences)
dataset["title_sequences"] = dataset["title"].apply(tokenizer.texts_to_sequences)


KeyboardInterrupt: 

In [ ]:
dataset["padded_text_seqs"] = dataset["text_sequences"].apply(pad_sequences(padding='post'))
dataset["padded_title_seqs"] = dataset["title_sequences"].apply(pad_sequences(padding='post'))